In [75]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
%matplotlib inline
pd.set_option('display.max_columns', None)

In [76]:
# Columns from kaggle dataset
old_col= '''Index(['Loan ID', 'Customer ID', 'Loan Status', 'Current Loan Amount', 'Term',
       'Credit Score', 'Annual Income', 'Years in current job',
       'Home Ownership', 'Purpose', 'Monthly Debt', 'Years of Credit History',
       'Months since last delinquent', 'Number of Open Accounts',
       'Number of Credit Problems', 'Current Credit Balance',
       'Maximum Open Credit', 'Bankruptcies', 'Tax Liens'],
      dtype='object')'''

# Read in CSV

In [77]:
#df = pd.read_csv('~/Downloads/LoanStats3c.csv',header = 1)
#df.to_pickle('./data/loan_data_2014.zip',compression = 'zip')

# Drop unnecessary columns:

In [96]:
df = pd.read_pickle('./data/loan_data_2014.zip',compression = 'zip')
df.shape

(235631, 145)

In [97]:
# add categorical variable - HARdship END FLAG EXISTS
# THIS EXISTS AS HARDSHIP_STATUS

In [98]:
df=df.drop(columns = [
        # Only looking at loans for individuals, not joint
        'annual_inc_joint',
        'application_type',
        'dti_joint',
        'verification_status_joint',
        'revol_bal_joint', 


        # Don't need specific ID info
        'id',
        'member_id',
    
        # Only care about information available pre loan
        'collection_recovery_fee',
        'collections_12_mths_ex_med',
        
        # Hard to classify, may use regex text processing in future
        'emp_title',
    
        # Keeping the hardship status but looking at pre-loan metrics only
        'hardship_flag',
        'hardship_type',
        'hardship_reason',
        'deferral_term',
        'hardship_amount',
        'hardship_start_date',
        'hardship_end_date',
        'payment_plan_start_date',
        'hardship_length',
        'hardship_dpd',
        'hardship_loan_status',
        'orig_projected_additional_accrued_interest',
        'hardship_payoff_balance_amount',
        'hardship_last_payment_amount',
    
        'last_pymnt_amnt',
        'last_pymnt_d',
        'next_pymnt_d',
        'pymnt_plan',
        'total_pymnt',
        'total_pymnt_inv',
    
        # Trying to extrapolate beyond certain issuing dates of loans
        'issue_d',
        'mths_since_last_major_derog',
        'mths_since_last_record',
        'num_tl_30dpd',
        'num_tl_120dpd_2m',
        
        'debt_settlement_flag',
        'debt_settlement_flag_date',
        'desc',
        
        # sub_grade category already contains grade
        'grade',
    
    
        # This is for investors / loan funding
        'initial_list_status',
        'funded_amnt',
        'funded_amnt_inv',              
        # Probably useful metrics, but all NaN in this dataset              
        'open_acc_6m',                 
        'open_il_12m',
        'open_il_24m',                     
        'open_act_il',   
        'open_rv_12m',
        'open_rv_24m',
                      
        'out_prncp', 
        'out_prncp_inv',
        'policy_code',
        'recoveries',
        # Columns related to debt settlement; we only care about pre loan features              
        'settlement_status',
        'settlement_date',
        'settlement_amount',
        'settlement_percentage',
        'settlement_term',
        # Only looking for one applicant 
        #'sec_app_fico_range_low', 
        #'sec_app_fico_range_high', 
        'sec_app_earliest_cr_line', 
        'sec_app_inq_last_6mths', 
        'sec_app_mort_acc', 
        'sec_app_open_acc', 
        'sec_app_revol_util',
        'sec_app_open_act_il',
        'sec_app_num_rev_accts',
        'sec_app_chargeoff_within_12_mths',
        'sec_app_collections_12_mths_ex_med', 
        'sec_app_mths_since_last_major_derog', 
        'title',
        'total_cu_tl',
                      

        'url',
    
        'zip_code'])

print('Before dropping NaN's: ',df.shape)
df.dropna(how='all', inplace = True)
print('After dropping NaN Rows: ',df.shape)
pre_drop_col = df.columns
df.dropna(axis=1, how='all',inplace = True)
print('After dropping NaN Columns: ',df.shape)
post_drop_col = df.columns

Before dropping NaN Rows:  (235631, 74)
After dropping NaN Rows:  (235629, 74)
Before dropping NaN Columns:  (235629, 74)
After dropping NaN Columns:  (235629, 67)


In [104]:
dropped_nan_columns = set(pre_drop_col) - set(post_drop_col)
print("Dropped NaN Columns: ", dropped_nan_columns)

Dropped NaN Columns:  {'mths_since_rcnt_il', 'inq_fi', 'inq_last_12m', 'max_bal_bc', 'all_util', 'il_util', 'total_bal_il'}


In [82]:
df.sample(5)

,loan_amnt,term,int_rate,installment,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,total_rec_prncp,total_rec_int,total_rec_late_fee,last_credit_pull_d,acc_now_delinq,tot_coll_amt,tot_cur_bal,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_status,disbursement_method
86047,9500.0,36 months,11.67%,314.05,B4,10+ years,RENT,70000.0,Not Verified,Fully Paid,credit_card,FL,2.19,0.0,May-2005,0.0,NaN,3.0,0.0,4189.0,54.4%,7.0,9500.00,1768.54,0.0,Mar-2019,0.0,0.0,4189.0,NaN,NaN,NaN,NaN,NaN,7700.0,NaN,NaN,3.0,1396.0,11.0,99.7,0.0,0.0,105.0,111.0,1.0,1.0,0.0,13.0,NaN,23.0,NaN,0.0,2.0,2.0,2.0,3.0,2.0,3.0,5.0,2.0,3.0,0.0,1.0,100.0,100.0,0.0,0.0,7700.0,4189.0,4200.0,0.0,NaN,Cash
5864,5000.0,36 months,8.19%,157.13,A5,3 years,MORTGAGE,96000.0,Not Verified,Fully Paid,other,IN,26.28,0.0,Sep-1992,0.0,NaN,13.0,0.0,49174.0,79.1%,22.0,5000.00,651.74,0.0,Sep-2017,0.0,0.0,206195.0,NaN,NaN,NaN,NaN,NaN,62200.0,NaN,NaN,1.0,15861.0,5904.0,88.7,0.0,0.0,183.0,266.0,80.0,7.0,2.0,84.0,NaN,15.0,NaN,0.0,4.0,7.0,4.0,7.0,9.0,7.0,11.0,7.0,13.0,0.0,1.0,100.0,75.0,0.0,0.0,240968.0,91097.0,52400.0,51123.0,NaN,Cash
52181,14175.0,36 months,16.99%,505.31,D3,10+ years,MORTGAGE,55000.0,Source Verified,Charged Off,home_improvement,PA,17.52,1.0,Apr-2000,0.0,7.0,8.0,0.0,12409.0,72.6%,22.0,2561.17,1467.93,0.0,Feb-2017,0.0,0.0,254230.0,NaN,NaN,NaN,NaN,NaN,17100.0,NaN,NaN,3.0,31779.0,1958.0,85.1,0.0,0.0,113.0,174.0,1.0,1.0,4.0,1.0,7.0,5.0,7.0,0.0,3.0,4.0,4.0,10.0,4.0,5.0,13.0,4.0,8.0,0.0,2.0,86.4,75.0,0.0,0.0,277690.0,26050.0,13100.0,16000.0,NaN,Cash
133830,9875.0,36 months,14.99%,342.28,C5,< 1 year,RENT,110000.0,Not Verified,Charged Off,debt_consolidation,WA,28.17,0.0,Apr-1991,1.0,NaN,13.0,3.0,6715.0,50.9%,28.0,2564.36,1229.04,0.0,Oct-2016,0.0,955.0,105344.0,NaN,NaN,NaN,NaN,NaN,13200.0,NaN,NaN,6.0,9577.0,5661.0,40.4,0.0,0.0,278.0,63.0,1.0,1.0,0.0,1.0,NaN,3.0,NaN,0.0,4.0,6.0,4.0,4.0,21.0,6.0,7.0,6.0,13.0,0.0,3.0,100.0,50.0,0.0,3.0,139465.0,105344.0,9500.0,126265.0,NaN,Cash
177033,22500.0,36 months,10.99%,736.52,B2,10+ years,MORTGAGE,100000.0,Verified,Fully Paid,credit_card,CA,22.14,0.0,Oct-1997,1.0,NaN,17.0,0.0,37427.0,65.3%,42.0,22500.00,2238.62,0.0,Mar-2018,0.0,0.0,468307.0,NaN,NaN,NaN,NaN,NaN,57300.0,NaN,NaN,6.0,27547.0,7080.0,82.8,0.0,0.0,124.0,198.0,4.0,4.0,5.0,12.0,NaN,4.0,NaN,0.0,7.0,9.0,7.0,19.0,8.0,12.0,29.0,9.0,17.0,0.0,4.0,100.0,85.7,0.0,0.0,515941.0,90911.0,41100.0,71641.0,NaN,Cash


In [83]:
#df[df['num_tl_120dpd_2m'] >= 1][['num_tl_30dpd','num_tl_120dpd_2m','num_tl_90g_dpd_24m']]

In [84]:
# df[['int_rate','grade','sub_grade']].sort_values(by = 'int_rate')

# Clean Columns

In [105]:
def drop_percentage_signs(df, column_name):
    new_column_name = column_name + '_percent'
    df[new_column_name] = df[column_name].astype(str).str[:-1].astype(np.float64)
    return df.drop(columns = [column_name])

In [106]:
df.shape

(235629, 67)

In [107]:
#df['revol_util'].fillna('0%',inplace = True)
#df['revol_util'] =df['revol_util'].astype(str).str[:-1]


In [108]:
#df['revol_util'].sort_values()


# print('Before dropping NaN Rows: ',df.shape)
# print('After dropping NaN Rows: ',df.shape)

In [109]:
# Clean columns
# Convert loan terms into two categories 'short' and 'long'
df['term'] = df['term'].str.replace('36 months','short').str.replace('60 months','long').str.strip()

# Convert interest rate from percentage to float
df['int_rate'].dropna(axis = 0,how = 'any', inplace = True)
df=drop_percentage_signs(df,'int_rate')


# Fill Na's with 0.0% in revolving balance (unused revolving balance counts as NaN)
df['revol_util'].fillna(value = '0.0%',inplace = True)
df=drop_percentage_signs(df, 'revol_util')

In [110]:
df.shape

(235629, 67)

In [111]:
df.sample(5)

,loan_amnt,term,installment,sub_grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,total_acc,total_rec_prncp,total_rec_int,total_rec_late_fee,last_credit_pull_d,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_status,disbursement_method,int_rate_percent,revol_util_percent
209587,20000.0,long,529.77,E1,8 years,MORTGAGE,60000.0,Verified,Fully Paid,home_improvement,TN,6.14,0.0,Oct-1999,0.0,NaN,4.0,1.0,10814.0,11.0,20000.0,9117.15,0.0,Mar-2019,0.0,0.0,203030.0,11600.0,3.0,50758.0,580.0,88.4,0.0,0.0,NaN,150.0,12.0,9.0,3.0,12.0,NaN,10.0,NaN,0.0,2.0,3.0,2.0,2.0,0.0,3.0,8.0,3.0,4.0,0.0,2.0,100.0,50.0,1.0,0.0,208600.0,10814.0,5000.0,0.0,NaN,Cash,19.99,93.2
147132,19375.0,long,563.02,F3,7 years,MORTGAGE,54000.0,Verified,Fully Paid,medical,AZ,3.02,0.0,Aug-2000,0.0,52.0,3.0,0.0,0.0,31.0,19375.0,12282.94,0.0,Nov-2017,0.0,100.0,205113.0,1100.0,2.0,102557.0,NaN,NaN,0.0,0.0,155.0,165.0,19.0,8.0,6.0,77.0,52.0,9.0,52.0,0.0,0.0,0.0,1.0,3.0,11.0,2.0,14.0,0.0,3.0,0.0,1.0,93.5,NaN,0.0,0.0,208100.0,0.0,0.0,0.0,NaN,Cash,24.50,0.0
108260,25000.0,long,573.32,C2,10+ years,RENT,130000.0,Source Verified,Fully Paid,debt_consolidation,CA,17.95,0.0,Jun-2001,1.0,NaN,7.0,0.0,30266.0,21.0,25000.0,3340.86,0.0,Jan-2018,0.0,0.0,75496.0,59400.0,5.0,10785.0,8934.0,77.2,0.0,0.0,122.0,157.0,21.0,4.0,0.0,21.0,NaN,4.0,NaN,0.0,3.0,3.0,3.0,9.0,9.0,5.0,12.0,3.0,7.0,0.0,2.0,100.0,66.7,0.0,0.0,115234.0,75496.0,39200.0,55834.0,NaN,Cash,13.35,51.0
224801,21000.0,long,483.54,B5,10+ years,MORTGAGE,87360.0,Source Verified,Fully Paid,debt_consolidation,TN,10.76,0.0,May-1996,0.0,NaN,9.0,0.0,18884.0,21.0,21000.0,3717.07,0.0,Oct-2018,0.0,0.0,165163.0,45100.0,7.0,18351.0,19044.0,27.9,0.0,0.0,128.0,212.0,17.0,17.0,1.0,17.0,NaN,12.0,NaN,0.0,3.0,7.0,4.0,10.0,5.0,8.0,15.0,7.0,9.0,0.0,0.0,100.0,0.0,0.0,0.0,195328.0,18884.0,26400.0,0.0,NaN,Cash,13.53,41.9
204895,9450.0,short,329.03,C4,NaN,OWN,27265.0,Verified,Fully Paid,debt_consolidation,FL,11.97,0.0,Jul-1995,2.0,NaN,6.0,1.0,6712.0,40.0,9450.0,1903.38,0.0,Nov-2015,0.0,0.0,6712.0,10050.0,4.0,1342.0,2625.0,54.0,0.0,0.0,127.0,223.0,2.0,2.0,4.0,6.0,NaN,1.0,NaN,0.0,3.0,5.0,9.0,16.0,2.0,5.0,36.0,5.0,5.0,0.0,3.0,75.0,0.0,1.0,0.0,10050.0,6712.0,5250.0,0.0,NaN,Cash,15.31,67.0


# One hot encoding 'Months Since' Variables

- First do a pd.cut() to bin the data into categories.
- Rename these during the function call
- Then fill nans as another category
- Run a pd.get_dummies on the resulting column

In [69]:
df['mths_since_last_delinq'] = \
pd.cut(df['mths_since_last_delinq'],5,labels =
       ['0-3 years','3-6 years','6-9 years','9-12 years','12-15 years'
       ]).cat.add_categories('never').fillna('never') # 

df['mths_since_recent_bc'] = \
pd.cut(df['mths_since_recent_bc'],5,labels =
       ['0-10 years','10-20 years','20-30 years','30-40 years','40-50 years']
      ).cat.add_categories('never').fillna('never')# 

In [153]:
pd.cut(df['mths_since_recent_bc_dlq'],5).value_counts()

(-0.17, 34.0]     27678
(34.0, 68.0]      26312
(68.0, 102.0]      8225
(102.0, 136.0]       55
(136.0, 170.0]       11
Name: mths_since_recent_bc_dlq, dtype: int64

In [150]:
pd.cut(df['mths_since_recent_bc_dlq'],5,labels =
       ['0-10 years','10-20 years','20-30 years','30-40 years','40-50 years']
      ).cat.add_categories('never').fillna('never')# 


0         10-20 years
1               never
2               never
3               never
4          0-10 years
5         10-20 years
6         10-20 years
7               never
8               never
9               never
10              never
11         0-10 years
12              never
13         0-10 years
14         0-10 years
15         0-10 years
16        20-30 years
17              never
18        10-20 years
19              never
20              never
21              never
22              never
23              never
24        10-20 years
25              never
26              never
27              never
28              never
29              never
             ...     
235599          never
235600          never
235601          never
235602     0-10 years
235603          never
235604     0-10 years
235605          never
235606     0-10 years
235607    10-20 years
235608    10-20 years
235609          never
235610          never
235611          never
235612          never
235613    

In [45]:
df.loc[df['mths_since_recent_bc_dlq'].isna()][['loan_status','mths_since_recent_bc_dlq','mort_acc']]

,loan_status,mths_since_recent_bc_dlq,mort_acc
1,Fully Paid,NaN,0.0
2,Fully Paid,NaN,0.0
3,Charged Off,NaN,0.0
7,Charged Off,NaN,6.0
8,Fully Paid,NaN,5.0
9,Charged Off,NaN,0.0
10,Fully Paid,NaN,0.0
12,Charged Off,NaN,0.0
17,Fully Paid,NaN,0.0
19,Fully Paid,NaN,2.0


142269     0.0
65867      0.0
65865      0.0
65864      0.0
191991     0.0
116060     0.0
191992     0.0
159075     0.0
191993     0.0
65857      0.0
191994     0.0
191997     0.0
65854      0.0
191999     0.0
192000     0.0
65850      0.0
65849      0.0
192002     0.0
192005     0.0
192006     0.0
65868      0.0
65845      0.0
65869      0.0
65871      0.0
116051     0.0
191974     0.0
65888      0.0
191975     0.0
116052     0.0
65885      0.0
          ... 
71457     21.0
203426    21.0
147927    22.0
43278     22.0
146612    22.0
55256     23.0
82970     23.0
218720    23.0
145104    23.0
103854    24.0
207978    24.0
168594    24.0
151445    24.0
117599    24.0
176407    24.0
98930     25.0
172823    25.0
213088    25.0
162981    25.0
209987    25.0
10326     26.0
181049    27.0
145777    27.0
189437    27.0
228208    28.0
113706    31.0
216053    32.0
95062     34.0
44819     36.0
203993    37.0
Name: mort_acc, Length: 235629, dtype: float64

In [42]:
pd.get_dummies(df,drop_first = True)

,loan_amnt,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,total_acc,total_rec_prncp,total_rec_int,total_rec_late_fee,acc_now_delinq,tot_coll_amt,tot_cur_bal,mths_since_rcnt_il,total_bal_il,il_util,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,int_rate_percent,revol_util_percent,term_short,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,sub_grade_A2,sub_grade_A3,sub_grade_A4,sub_grade_A5,sub_grade_B1,sub_grade_B2,sub_grade_B3,sub_grade_B4,sub_grade_B5,sub_grade_C1,sub_grade_C2,sub_grade_C3,sub_grade_C4,sub_grade_C5,sub_grade_D1,sub_grade_D2,sub_grade_D3,sub_grade_D4,sub_grade_D5,sub_grade_E1,sub_grade_E2,sub_grade_E3,sub_grade_E4,sub_grade_E5,sub_grade_F1,sub_grade_F2,sub_grade_F3,sub_grade_F4,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year,home_ownership_MORTGAGE,home_ownership_OWN,home_ownership_RENT,verification_status_Source Verified,verification_status_Verified,loan_status_Current,loan_status_Default,loan_status_Fully Paid,loan_status_In Grace Period,loan_status_Late (16-30 days),loan_status_Late (31-120 days),purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,addr_state_AL,addr_state_AR,addr_state_AZ,addr_state_CA,addr_state_CO,addr_state_CT,addr_state_DC,addr_state_DE,addr_state_FL,addr_state_GA,addr_state_HI,addr_state_IA,addr_state_ID,addr_state_IL,addr_state_IN,addr_state_KS,addr_state_KY,addr_state_LA,addr_state_MA,addr_state_MD,addr_state_ME,addr_state_MI,addr_state_MN,addr_state_MO,addr_state_MS,addr_state_MT,addr_state_NC,addr_state_NH,addr_state_NJ,addr_state_NM,addr_state_NV,addr_state_NY,addr_state_OH,addr_state_OK,addr_state_OR,addr_state_PA,addr_state_RI,addr_state_SC,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,earliest_cr_line_Apr-1962,earliest_cr_line_Apr-1963,earliest_cr_line_Apr-1964,earliest_cr_line_Apr-1965,earliest_cr_line_Apr-1966,earliest_cr_line_Apr-1967,earliest_cr_line_Apr-1968,earliest_cr_line_Apr-1969,earliest_cr_line_Apr-1970,earliest_cr_line_Apr-1971,earliest_cr_line_Apr-1972,earliest_cr_line_Apr-1973,earliest_cr_line_Apr-1974,earliest_cr_line_Apr-1975,earliest_cr_line_Apr-1976,earliest_cr_line_Apr-1977,earliest_cr_line_Apr-1978,earliest_cr_line_Apr-1979,earliest_cr_line_Apr-1980,earliest_cr_line_Apr-1981,earliest_cr_line_Apr-1982,earliest_cr_line_Apr-1983,earliest_cr_line_Apr-1984,earliest_cr_line_Apr-1985,earliest_cr_line_Apr-1986,earliest_cr_line_Apr-1987,earliest_cr_line_Apr-1988,earliest_cr_line_Apr-1989,earliest_cr_line_Apr-1990,earliest_cr_line_Apr-1991,earliest_cr_line_Apr-1992,earliest_cr_line_Apr-1993,earliest_cr_line_Apr-1994,earliest_cr_line_Apr-1995,earliest_cr_line_Apr-1996,earliest_cr_line_Apr-1997,earliest_cr_line_Apr-1998,earliest_cr_line_Apr-1999,earliest_cr_line_Apr-2000,earliest_cr_line_Apr-2001,earliest_cr_line_Apr-2002,earliest_cr_line_Apr-2003,earliest_cr_line_Apr-2004,earliest_cr_line_Apr-2005,earliest_cr_line_Apr-2006,earli

In [ ]:
dt.datetime.strptime(test_date,'%b-%Y').date()

In [ ]:
df['loan_status'].unique()

# Patsy.dmatrix (pass string like last project, but no y~)